In [3]:
import numpy as np
import dask , dask.distributed
import xarray as xr
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import warnings
warnings.simplefilter('ignore')
import dask_jobqueue
from astropy.convolution import Box2DKernel, convolve
import xskillscore as xs

ModuleNotFoundError: No module named 'xskillscore'

# create monthly data

## year 0181

In [3]:
###load data
file_path='/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5/'
ds_fluxes=xr.open_dataset(file_path+'01810101.ocean_minibling_surf_flux.nc', chunks={"time": 10})[['o2_stf', 'dic_stf']]*60*60*24*365
ds_ssh_sst=xr.open_dataset(file_path+'01810101.ice_daily.nc', chunks={"time": 10})[['SSH', 'SST']]
ds_heat=xr.open_dataset(file_path+'01810101.ocean_bdy_flux.nc', chunks={"time": 10})[['sens_heat', 'evap_heat']]

###correct grid information in ice file
ds_ssh_sst=ds_ssh_sst.assign_coords({'xt': (ds_fluxes.xt_ocean.data), 
                                       'yt': (ds_fluxes.yt_ocean.data)})
ds_ssh_sst=ds_ssh_sst.rename({'xt': 'xt_ocean', 'yt': 'yt_ocean'})


In [4]:
#take monthly means
ds_fluxes_monthly=ds_fluxes.groupby('time.month').mean('time')
ds_ssh_sst_monthly=ds_ssh_sst.groupby('time.month').mean('time')
ds_heat_monthly=ds_heat.groupby('time.month').mean('time')

In [5]:
###save in one file

ds=xr.merge([ds_fluxes_monthly,ds_ssh_sst_monthly, ds_heat_monthly])
save_path='/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_concat/MOM5_daily2monthly.nc'
ds.to_netcdf(save_path)

## all months

# filter monthly data with boxfilter

## year 0181

In [6]:
def Boxfilter(data, size=30):
    #uses 30 data points in each direction to average by default--> for 0.1° resolution this is a 3°x3° filter
    kernel = Box2DKernel(size)
    #wrap assumes periodic boundaries --> for snippets, frame needs to be cut off
    conv=convolve(data, kernel, boundary='wrap')
    conv=data.copy(data=conv)
    return conv

In [7]:
def Data_3D(data, time='month'):
    #filters for every time step, need to specify time --> 'month', 'time'
    res=[]
    for date in data[time]:
        conv=Boxfilter(data.sel(data[time]=date), size=30)
        res.append(conv)
    da_res=xr.concat(res, dim=time)
    return da_res

In [ ]:
### apply filter
sst=Data_3D(ds.SST)
ssh=Data_3D(ds.SSH)
o2=Data_3D(ds.o2_stf)
dic=Data_3D(ds.dic_stf)
sh=Data_3D(ds.sens_heat)
lh=Data_3D(ds.evap_heat)

ds_ano=xr.merge([sst, ssh, o2, dic, sh, lh])

In [4]:
ds_ano.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/3x3_boxfilter_monthly.nc')

# save anomalies in an extra file

In [6]:
ds=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/MOM5_daily2monthly/MOM5_daily2monthly.nc')
ds_ano=xr.open_dataset('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/3x3_boxfilter_monthly.nc')

In [7]:
dsa=ds-ds_ano

In [9]:
dsa.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/3x3_boxfilter_anomaly_monthly.nc')

# correlations

In [2]:
pathf='/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/'
dsa=xr.open_dataset(pathf+'3x3_boxfilter_anomaly_monthly.nc')

In [3]:
sh=dsa.sens_heat
lh=dsa.evap_heat
o2=dsa.o2_stf
dic=dsa.dic_stf
ssh=dsa.SSH
sst=dsa.SST

In [4]:
ssh_sst=xr.corr(ssh, sst, dim='month')
ssh_sst.name='corr_sst_ssh'
sst_sh=xr.corr(sst, sh, dim='month')
sst_sh.name='corr_sst_sh'
sst_lh=xr.corr(sst, lh, dim='month')
sst_lh.name='corr_sst_lh'
sst_o2=xr.corr(sst, o2, dim='month')
sst_o2.name='corr_sst_o2'
sst_dic=xr.corr(sst, dic, dim='month')
sst_dic.name='corr_sst_dic'

In [5]:
correlations=xr.merge([ssh_sst,sst_o2, sst_dic, sst_sh, sst_lh])
correlations.to_netcdf('/gxfs_work/geomar/smomw577/mesoscale_eddies/BOX_filtered/3x3_boxfilter_sst_correlations_monthly.nc')

In [ ]:
#### pvalue

In [5]:
ssh_sst_p=xs.pearson_r_eff_p_value(ssh, sst, dim=None, skipna=False, keep_attrs=False)

NameError: name 'xskillscore' is not defined